In [1]:
import pickle
with open("PC_ms1.db",'rb') as f:
    PC_db=pickle.load(f)
    PC_db['PRECURSORMZ']=PC_db['PRECURSORMZ'].apply(float)

In [16]:
import pymzml
import numpy as np
from functools import reduce
def get_PC_ms1(file,delta=10e-6):
    TIC=pymzml.run.Reader(file)
    pc_num=0
    total_num=TIC.get_spectrum_count()
    spect_df=pd.DataFrame(columns=['m/z','inten','sid','low_win','high_win'])
    
    '''###
    spec1=TIC[1]
    pority=spec1['MS:1000129']
    print(pority,type(pority))
    '''###
    
    #return 

    for spectrum in TIC:
        #print(spectrum.ID,type(spectrum.ID))
        #print(f"handling the {spectrum.ID}th /{total_num} spectrum")
        if spectrum['MS:1000129'] or spectrum['MS:1000511']==1:
            continue
            
            
        if spectrum.has_peak(184.07294):
            current_mz=spectrum['MS:1000744']
            current_inten=spectrum['MS:1000042']
            current_sid=spectrum.ID
            current_low_win  =current_mz-spectrum['MS:1000828']
            current_hight_win=current_mz+spectrum['MS:1000829']
            output=[current_mz,current_inten,current_sid,current_low_win,current_hight_win]
            #current_pority=1 if spec1['MS:1000130'] else 2
            if spect_df.empty:
                spect_df.loc[len(spect_df),:]=output
                pc_num+=1
                continue
            if  abs( (spect_df['m/z']-current_mz)/current_mz).min()>delta:
                spect_df.loc[len(spect_df),:]=output
                pc_num+=1
            else:#update
                hit=(abs(spect_df['m/z']-current_mz) == abs(spect_df['m/z']-current_mz).min())
                
                #print(hit,f'\n the hit num is {hit.sum()}\n',spect_df.loc[hit,:])
                if (spect_df.loc[hit,'inten']<current_inten).all():
                    spect_df.loc[hit,:]=output
    return spect_df

def get_standerd_ms1(file,delta=5e-6,measured_precision=5e-6):
    TIC=pymzml.run.Reader(file)
    pc_num=0
    total_num=TIC.get_spectrum_count()
    #TIC[0]
    #return 
    ms1_df=pd.DataFrame(columns=['sid','inten'])
    for spectrum in TIC:
        if spectrum['MS:1000130'] and spectrum['MS:1000511']==1:
            ms1_df.loc[len(ms1_df),:]=[spectrum.ID,spectrum['MS:1000285']]
    #print(ms1_df)
    
    inten=ms1_df['inten'].values
    hit= (ms1_df['inten']> np.percentile(inten,80)) & (ms1_df['inten']<= np.percentile(inten,90))
    wanted=ms1_df.loc[hit,:]
    spectrum=pymzml.spec.Spectrum()
    for item in wanted['sid']:
        spectrum=spectrum+TIC[item]
    spectrum=spectrum/len(wanted)
    spectrum.peaks('reprofiled')
    spectrum.measured_precision=measured_precision
    spectrum.base_peak_inten=np.array(spectrum.peaks('reprofiled')).max(axis=0)[1]#centroided
    #print(spectrum.peaks('reprofiled'))
    return spectrum
def indentify_mz_with_db(df,delta=5e-6):
    def deal(mz):
        #print(mz,type(mz))
        err_count=abs(PC_db['PRECURSORMZ']-mz)/mz
        hit= err_count <= delta
        if hit.sum()>0:
            name=PC_db.loc[err_count==err_count.min(),:].iloc[0,:]["NAME"]
            return name
        else:
            return None
    temp_df_in_indentify=copy.deepcopy(df)
    temp_df_in_indentify['NAME']=temp_df_in_indentify['m/z'].apply(deal)
    
    #temp_df_in_indentify.drop_duplicates(subset=[])
    #去重，保留inten大的重复值
    identified_df_output=pd.DataFrame(columns=temp_df_in_indentify.columns)
    for idx,ser in temp_df_in_indentify.iterrows():
        hit=identified_df_output['NAME']==ser['NAME']
        if hit.sum()>0:
            hit_item=identified_df_output.iloc[hit.argmax(),:]
            if hit_item['inten']<ser['inten']:
                identified_df_output.iloc[hit.argmax(),:]=ser
        
        else:
            identified_df_output.loc[len(identified_df_output),:]=ser
    
    
    return identified_df_output
def indentify_mz_with_db2(df,delta=10e-6):
    def deal(ser):
        mz=ser['m/z']
        #print(mz,type(mz))
        #outlet=[name,error,ion_type]
        outlet=[None,None,None]
        err_count=abs(PC_db['PRECURSORMZ']-mz)/mz
        hit= err_count <= delta
        if hit.sum()>0:
            min_error=err_count.min()
            got_item=PC_db.loc[err_count==min_error,:].iloc[0,:]
            name=got_item["NAME"]
            outlet[0]=name
            outlet[1]=min_error*1e6
            outlet[2]=got_item["PRECURSORTYPE"]
            return outlet
        else:
            return outlet
    temp_df_in_indentify=copy.deepcopy(df)
    temp_df_in_indentify[['NAME','error','type']]=temp_df_in_indentify.apply(deal,result_type='expand',axis=1)
    return temp_df_in_indentify
def from_standerd_spectrum_get_inten(standerd_spectrum,is_absolute_inten=True):
    local_standerd_spectrum=standerd_spectrum
    #print("        current spectrum's base inten is:",local_standerd_spectrum.base_peak_inten)
    def deal(ser):
        #print(ser)
        mz=ser['m/z']
        inten=ser['inten']
        hit=local_standerd_spectrum.has_peak(mz)
        
        if len(hit)>0:
            if is_absolute_inten:
                return np.array(hit).max(axis=0)[1]
            else:
                return np.array(hit).max(axis=0)[1]/local_standerd_spectrum.base_peak_inten
        else:
            print(mz,' not found')
            return 0
    return deal
def merge_groups_meth2(*dfs):
    #单纯的鉴定结果联合
    output=pd.DataFrame(columns=dfs[0].columns)
    for temp_df in dfs:
        for idx,item in temp_df.iterrows():
            name_bits=output['NAME']==item['NAME']
            type_bits=output['type']==item['type']
            if name_bits.sum()>0:
                
                #完全一样,找偏差小的
                got=(name_bits & type_bits)
                if got.sum()>0:
                    if output.iloc[got.argmax(),:]['error']>item['error']:
                        output.iloc[got.argmax(),:]=item
                
                else:#类型不一样，优先保留加H
                    if item['type']=='[M+H]+':
                        output.loc[name_bits.argmax(),:]=item
            
            else:#m没有重复，直接添加
                #print('success add')
                output.loc[len(output),:]=item
            
    return output
def merge_groups(*dfs):
    #result=pd.DataFrame()
    dfs=copy.deepcopy(dfs)
    try:
        [x.set_index('NAME',inplace=True) for x in dfs]
        for col in ['index','inten','m/z']:
            try:
                [x.drop([col],axis=1, inplace=True) for x in dfs]
            except Exception as e:
                pass
    except Exception as e:
        pass
    
    res_merge= reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True), dfs)
    return  res_merge
def filter_columns(df_input,therhold=0.3):
    pass
    inten_columns=[x for x in df_input.columns if "mzML" in x]
    hit=(df_input.loc[:,inten_columns]>0).sum(axis=1) > therhold*len(inten_columns)
    #print(hit)
    output_df=copy.deepcopy(df_input.loc[hit,:] )
    return output_df

In [17]:
import pandas as pd
import pickle
import re
import numpy as np
import os
import copy
def get_all_group_pc(para,pricession=10e-5):
    #### PC 确定的时候，有可能是因为没有采集到二级而鉴定失败，所以再排除不是二级的质谱图时，以全部组别混合的二级PC进行排除 stage I 获得全组的PC二级
    pricession=10e-5
    all_group_pc=pd.DataFrame()
    for idx,content in para.iterrows():
        #if content['group']!=group_id:continue
        mzml_file=os.path.join(content['dir'],content['files'])
        
        current_file_pc_df=get_PC_ms1(mzml_file)
        current_file_pc_df['group']=content['group']
        print(f"the current mzml_file is{os.path.split(mzml_file)[1]},the len of  current_file_pc_df is {len(current_file_pc_df)}")
        if all_group_pc.empty:
            print("        it is the frist mzml")
            all_group_pc=current_file_pc_df
        else:
            for j_idx,j_content in current_file_pc_df.iterrows():
                hit=abs(all_group_pc['m/z']-j_content['m/z'])/j_content['m/z']<=pricession
                if hit.sum()==0:
                    all_group_pc=all_group_pc.append(j_content)

        print(f"        contained the {idx} file , the len of group_pc is {len(all_group_pc)}" )
    print(all_group_pc.shape)
    return all_group_pc
def all_groups_pc_for_get_group_result(group_id,all_group_pc,para,measured_precision=10e-5):
    #stage I
    group_id=group_id
    measured_precision=measured_precision
    group_peak=pd.DataFrame()
    for idx,content in para.iterrows():
        if content['group']!=group_id:continue
        mzml_file=os.path.join(content['dir'],content['files'])
        standerd_ms1=get_standerd_ms1(mzml_file)
        standerd_ms1.measured_precision=measured_precision
        current_peaks_df=pd.DataFrame(standerd_ms1.peaks('centroided'),columns=['m/z','inten'])

        if group_peak.empty:
            group_peak=current_peaks_df
        else:
            for j_idx,j_content in current_peaks_df.iterrows():
                #print(group_peak)
                #print(j_content)
                hit=abs(group_peak['m/z']-j_content['m/z'])/j_content['m/z']<=measured_precision
                if hit.sum()==0:
                    group_peak=group_peak.append(j_content)
    print(group_peak.shape)
    group_peak



    ##stage III
    group_peak_pc_df=pd.DataFrame(columns=group_peak.columns)
    for idx,content in group_peak.iterrows():
        #print(group_peak_pc_df)
        #print(content)
        hit= (all_group_pc['low_win']<= content['m/z']) & (all_group_pc['high_win']>= content['m/z'])  
        if hit.sum()>0:
            group_peak_pc_df.loc[len(group_peak_pc_df),:]=content
    print(group_peak_pc_df.shape)
    group_peak_pc_df

    ##stage IV 
    identified_df_group_m2_group1=indentify_mz_with_db(group_peak_pc_df,measured_precision).dropna()
    print(identified_df_group_m2_group1.shape)
    identified_df_group_m2_group1

    ##stage V
    for idx,content in para.iterrows():
        if content['group']!=group_id:continue
        mzml_file=os.path.join(content['dir'],content['files'])
        standerd_ms_temp=get_standerd_ms1(mzml_file)
        identified_df_group_m2_group1[content['files']]=identified_df_group_m2_group1.apply(from_standerd_spectrum_get_inten(standerd_ms_temp,False),axis=1)
    identified_df_group_m2_group1
    
    
    # 不要碳链长度
    return identified_df_group_m2_group1
def all_groups_pc_for_get_all_PC_indentified_result(group_id,all_group_pc,para,measured_precision=5e-5):
    group_id=group_id
    measured_precision=measured_precision
    group_peak=pd.DataFrame()
    for idx,content in para.iterrows():
        if content['group']!=group_id:continue
        mzml_file=os.path.join(content['dir'],content['files'])
        standerd_ms1=get_standerd_ms1(mzml_file)
        standerd_ms1.measured_precision=measured_precision
        current_peaks_df=pd.DataFrame(standerd_ms1.peaks('centroided'),columns=['m/z','inten'])

        if group_peak.empty:
            group_peak=current_peaks_df
        else:
            for j_idx,j_content in current_peaks_df.iterrows():
                #print(group_peak)
                #print(j_content)
                hit=abs(group_peak['m/z']-j_content['m/z'])/j_content['m/z']<=measured_precision
                if hit.sum()==0:
                    group_peak=group_peak.append(j_content)
    print(group_peak.shape)
    group_peak



    ##stage III
    group_peak_pc_df=pd.DataFrame(columns=group_peak.columns)
    for idx,content in group_peak.iterrows():
        #print(group_peak_pc_df)
        #print(content)
        hit= (all_group_pc['low_win']<= content['m/z']) & (all_group_pc['high_win']>= content['m/z'])  
        if hit.sum()>0:
            group_peak_pc_df.loc[len(group_peak_pc_df),:]=content
    print(group_peak_pc_df.shape)
    group_peak_pc_df

    ##stage IV 
    identified_df_group_m2_group1=indentify_mz_with_db2(group_peak_pc_df,measured_precision).dropna()
    print(identified_df_group_m2_group1.shape)
    identified_df_group_m2_group1

    
    
    # 不要碳链长度
    return identified_df_group_m2_group1

In [18]:
def demo_deal(raw_file_path):
    import pandas as pd
    para=pd.read_excel(raw_file_path)
    all_group_pc=get_all_group_pc(para,pricession=10e-5)
    all_group_dfs=[]
    for group_id in para['group'].unique():
        all_group_dfs.append(all_groups_pc_for_get_group_result(group_id,all_group_pc,para,measured_precision=10e-5))

    result_merge=merge_groups(*all_group_dfs).fillna(0)
    
    
    filter_df=filter_columns(result_merge,0.4)
    filter_df.to_excel('for_pic.xlsx')
demo_deal(r'./mzML_files.xlsx')

the current mzml_file is20221014_iPicoESI_APCI_A549_P&N_lipid_01.mzML,the len of  current_file_pc_df is 141
        it is the frist mzml
        contained the 0 file , the len of group_pc is 141
the current mzml_file is20221014_iPicoESI_APCI_HCCLM_P&N_lipid_24.mzML,the len of  current_file_pc_df is 122
        contained the 1 file , the len of group_pc is 199
(199, 6)
(1487, 2)
(375, 2)
(192, 3)
(1127, 2)
(322, 2)
(181, 3)
          20221014_iPicoESI_APCI_A549_P&N_lipid_01.mzML  \
NAME                                                      
PC 17:0                                        0.052482   
SM 20:3                                        0.000303   
PC 27:0                                        0.001577   
PC 26:3                                        0.008134   
SM 31:5                                        0.002457   
...                                                 ...   
SM 68:13                                       0.013299   
PC 66:18                                  